In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
from bs4 import BeautifulSoup
import requests

In [33]:
record_path_local = '../../../2_data/record_path_mimic3_matched.csv'
record_root_url = 'https://physionet.org/content/mimic3wdb/1.0/matched'
record_path_url = record_root_url + '/RECORDS-waveforms'

signal_extract_name = 'PLETH'
signal_extract_minimum_length = 5 # the minimum length of the signal segment, in minutes

#record_root_url = 'https://archive.physionet.org/physiobank/database/mimic3wdb'
#record_path_url = record_root_url + '/RECORDS'

print('record_path_local: {}'.format(record_path_local))
print('record_root_url: {}'.format(record_root_url))
print('record_path_url: {}'.format(record_path_url))

record_path_local: ../../../2_data/record_path_mimic3_matched.csv
record_root_url: https://physionet.org/content/mimic3wdb/1.0/matched
record_path_url: https://physionet.org/content/mimic3wdb/1.0/matched/RECORDS-waveforms


In [4]:
print('https://physionet.org/content/mimic3wdb/1.0/matched/RECORDS-waveforms')
'''
p00/p000033/p000033-2116-12-24-12-35
'''

https://physionet.org/content/mimic3wdb/1.0/matched/RECORDS-waveforms


'\np00/p000033/p000033-2116-12-24-12-35\n'

In [5]:
print('https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000033/p000033-2116-12-24-12-35.hea')
'''
p000033-2116-12-24-12-35/6 2 125 10102500 12:35:06.147 24/12/2116
3713820_layout 0
3713820_0001 15000
~ 219
3713820_0002 3379563
3713820_0003 6700437
~ 7281
'''

https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000033/p000033-2116-12-24-12-35.hea


'\np000033-2116-12-24-12-35/6 2 125 10102500 12:35:06.147 24/12/2116\n3713820_layout 0\n3713820_0001 15000\n~ 219\n3713820_0002 3379563\n3713820_0003 6700437\n~ 7281\n'

In [6]:
print('https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000033/3713820_0001.hea')
'''
3713820_0001 2 125 15000 12:35:06.147
3713820_0001.dat 80 24/mV 8 0 0 9875 0 II
3713820_0001.dat 80 24/mV 8 0 0 -1750 0 MCL1
'''

https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000033/3713820_0001.hea


'\n3713820_0001 2 125 15000 12:35:06.147\n3713820_0001.dat 80 24/mV 8 0 0 9875 0 II\n3713820_0001.dat 80 24/mV 8 0 0 -1750 0 MCL1\n'

In [7]:
def get_url_content(url, tag=None):
    
    '''
    This function scrape a list of useful information from a given PhysioNet URL.
    If the URL address points to an HTML document, the information to be extracted is define by a tag.
    I found this address (https://hackersandslackers.com/scraping-urls-with-beautifulsoup/) usefull
      for details on how to scrape a web page
    '''
    
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }    
    
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    if '<!DOCTYPE html>' in str(soup):
        content = getattr(soup, str(tag)).getText()
        #content = soup.pre.getText()
    else:
        content = soup.getText()
    return content.splitlines()

In [10]:
tag = 'pre'
if os.path.exists(record_path_local):
    record_path_list = pd.read_csv(record_path_local, header=None)
    record_path_list = list(record_path_list.iloc[:,0])
    print('baubau_1')
else:
    target_url = record_path_url
    record_path_list = get_url_content(target_url, tag)
    pd.DataFrame(record_path_list).to_csv(record_path_local, header=None, index=None)
    print('baubau_2')

baubau_2


In [39]:
# TO DO:
# we have to decide whether the index_of_last_record_processed is 0 or 1 based 
#  (i.e. it starts from zero or from one)

tag = 'pre'
index_of_last_record_processed = -1
for i in range(index_of_last_record_processed + 1, 1):
    print(i)
    record_path = record_path_list[i]
    print('record_path: {}'.format(record_path))
    
    #directory p04/p044083.
    directory = str(record_path.split("/")[0]) + '/' + str(record_path.split("/")[1])
    print('directory: {}'.format(directory))
    
    intermediate_directory = str(record_path.split("/")[0])
    print('intermediate_directory: {}'.format(intermediate_directory))
    subject_id = record_path.split("/")[1]
    subject_id = int(subject_id[1:])
    print('subject_id: {}'.format(subject_id))
    #record_directory = str(record_path.split("/")[1])
    #print(record_directory)
    
    #print('https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000033/p000033-2116-12-24-12-35.hea')
    #print(record_root_url + '/' + record_path + '.hea')
    #print('https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000020/p000020-2183-04-28-17-47.hea')
    #print('https://physionet.org/content/mimic3wdb-matched/1.0/p00/p000020/p000020-2183-04-28-17-47.hea')
    
    master_waveform_header = record_root_url + '/' + record_path + '.hea'
    print('master_waveform_header: {}'.format(master_waveform_header))
    
    target_url = master_waveform_header
    master_waveform_content = get_url_content(target_url, tag)
    print('master_waveform_content: {}'.format(master_waveform_content))
    
    print('       ...........\n')
    
    master_waveform_content_first_line = master_waveform_content[0].split(' ')
    record_length = int(master_waveform_content_first_line[3]) / int(master_waveform_content_first_line[2])
    print('record_length (s): {}'.format(record_length))
    
    print('       ...........\n')    
        
    waveform_layout_header = master_waveform_content[1].split(' ')[0]
    waveform_layout_header = waveform_layout_header + '.hea'
    waveform_layout_header = record_root_url + '/' + directory + '/' + waveform_layout_header
    print('waveform_layout_header: {}'.format(waveform_layout_header))    
    target_url = waveform_layout_header
    waveform_layout_content = get_url_content(target_url, tag)
    print('waveform_layout_content: {}'.format(waveform_layout_content))
        
    signal_names = [x.split(' ')[-1] for x in waveform_layout_content[1:-1]]
    print('signal_names: {}'.format(signal_names))
    
    if signal_extract_name in signal_names and record_length > 60 * signal_extract_minimum_length: # and len(record_segments_path_list):
        #for record_segment in record_segments_path_list:
        #    print('record_segment: {}'.format(record_segment))
        break

    
    # https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000107/p000107-2121-11-30-20-03.hea
    
    
    print('       ...........\n')
    index_of_last_record_processed = index_of_last_record_processed + 1
    print('index_of_last_record_processed: {}'.format(index_of_last_record_processed))    
    print('--------------------------------------')

    
    



0
record_path: p00/p000020/p000020-2183-04-28-17-47
directory: p00/p000020
intermediate_directory: p00
subject_id: 20
master_waveform_header: https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000020/p000020-2183-04-28-17-47.hea
master_waveform_content: ['p000020-2183-04-28-17-47/10 4 125 9862593 17:47:59.486 28/04/2183', '3544749_layout 0', '~ 93', '3544749_0001 3811', '3544749_0002 124', '3544749_0003 8', '3544749_0004 380', '3544749_0005 7098177', '3544749_0006 15000', '3544749_0007 30000', '3544749_0008 2715000']
       ...........

record_length (s): 78900.744
       ...........

waveform_layout_header: https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000020/3544749_layout.hea
waveform_layout_content: ['3544749_layout 4 125 0 17:47:59.486', '~ 0 43/mV 14 0 -8192 0 0 II', '~ 0 50/mV 14 0 -8192 0 0 AVF', '~ 0 1.25/mmHg 9 0 -256 0 0 ABP', '~ 0 2.5/mmHg 10 0 -512 0 0 PAP']
signal_names: ['II', 'AVF', 'ABP']
       ...........

index_of_last_record_processed: 0
----------

In [42]:
master_waveform_content

['p000020-2183-04-28-17-47/10 4 125 9862593 17:47:59.486 28/04/2183',
 '3544749_layout 0',
 '~ 93',
 '3544749_0001 3811',
 '3544749_0002 124',
 '3544749_0003 8',
 '3544749_0004 380',
 '3544749_0005 7098177',
 '3544749_0006 15000',
 '3544749_0007 30000',
 '3544749_0008 2715000']

In [115]:
# get the waveform record name
waveform_record_name = [i.split(' ')[0].split('_') for i in master_waveform_content if '3' == i[0]][0][0]
waveform_record_name

'3544749'

In [119]:
# get the waveform_record_state
waveform_record_state = [(waveform_record_name + '_' in x) and not (waveform_record_name + '_layout' in x) 
                for x in master_waveform_content]
print(waveform_record_state)

[False, False, False, True, True, True, True, True, True, True, True]


In [121]:
# get the waveform_record_segment
waveform_record_segment = [master_waveform_content[i].split(' ') for i, j in enumerate(waveform_record_state) if j]
waveform_record_segment

[['3544749_0001', '3811'],
 ['3544749_0002', '124'],
 ['3544749_0003', '8'],
 ['3544749_0004', '380'],
 ['3544749_0005', '7098177'],
 ['3544749_0006', '15000'],
 ['3544749_0007', '30000'],
 ['3544749_0008', '2715000']]

In [104]:
[j[0] for j in record_segment if 
 int(j[1]) / int(master_waveform_content_first_line[2]) > 60 * signal_extract_minimum_length]

['3544749_0005', '3544749_0008']

In [90]:
[record_segment_info for index, record_segment_info in enumerate(record_segment)  ]

['3544749_0001 3811',
 '3544749_0002 124',
 '3544749_0003 8',
 '3544749_0004 380',
 '3544749_0005 7098177',
 '3544749_0006 15000',
 '3544749_0007 30000',
 '3544749_0008 2715000']

In [70]:
for index, i in zip(range(len(record_state)), record_state):
    if i:
        record_segment = master_waveform_content[index]
        #print(index, i, record_segment)
        segment_length = int(record_segment.split(' ')[1]) / int(master_waveform_content_first_line[2])
        if  segment_length > 60 * signal_extract_minimum_length:
            #print('bingo')
            print(record_segment.split(' ')[0])

3544749_0005
3544749_0008


In [67]:
int(master_waveform_content_first_line[2])

125

In [66]:
record_segment

'3544749_0008 2715000'

In [63]:
record_segment = [index, i in zip(range(len(record_state)), record_state)]

In [64]:
record_segment

[10, False]

In [55]:
# initializing list
test_list = [1, 4, 5, 6, 7]
 
# Printing list
print("Original list is : " + str(test_list))
 
# using zip() to
# get index and value
print("List index-value are : ")
for index, value in zip(range(len(test_list)), test_list):
    print(index, value)

Original list is : [1, 4, 5, 6, 7]
List index-value are : 
0 1
1 4
2 5
3 6
4 7


In [52]:
# res_list = [test_list[i] for i in index_list]
int(False)

0

In [49]:
master_waveform_content[0,1]

TypeError: list indices must be integers or slices, not tuple

In [37]:
print('https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000107/3168852_0001.hea')
print('3168852_0001 3 125 6250 05:53:26')
print('3168852_0001.dat 80 128(-65)/mV 8 0 -61 2696 0 II')
print('3168852_0001.dat 80 128(-65)/mV 8 0 -26 -31263 0 V')
print('3168852_0001.dat 80 1.28(-109)/mmHg 8 0 -109 -25890 0 ABP')

https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000107/3168852_0001.hea
3168852_0001 3 125 6250 05:53:26
3168852_0001.dat 80 128(-65)/mV 8 0 -61 2696 0 II
3168852_0001.dat 80 128(-65)/mV 8 0 -26 -31263 0 V
3168852_0001.dat 80 1.28(-109)/mmHg 8 0 -109 -25890 0 ABP


In [21]:
master_waveform_content

['p000020-2183-04-28-17-47/10 4 125 9862593 17:47:59.486 28/04/2183',
 '3544749_layout 0',
 '~ 93',
 '3544749_0001 3811',
 '3544749_0002 124',
 '3544749_0003 8',
 '3544749_0004 380',
 '3544749_0005 7098177',
 '3544749_0006 15000',
 '3544749_0007 30000',
 '3544749_0008 2715000']

In [38]:
master_waveform_content

['p000107-2122-05-14-21-01/46 5 125 20101375 21:01:58.000 14/05/2122',
 '3860035_layout 0',
 '3860035_0001 739500',
 '3860035_0002 750',
 '3860035_0003 125',
 '3860035_0004 1928625',
 '3860035_0005 125',
 '3860035_0006 2815000',
 '3860035_0007 9875',
 '3860035_0008 32500',
 '3860035_0009 157250',
 '3860035_0010 53875',
 '3860035_0011 47125',
 '3860035_0012 172875',
 '3860035_0013 10125',
 '3860035_0014 96750',
 '3860035_0015 7625',
 '3860035_0016 45375',
 '3860035_0017 2000',
 '3860035_0018 4750',
 '3860035_0019 13000',
 '3860035_0020 26250',
 '3860035_0021 1625',
 '3860035_0022 291250',
 '3860035_0023 6000',
 '3860035_0024 138000',
 '3860035_0025 1875',
 '3860035_0026 1500',
 '3860035_0027 36000',
 '3860035_0028 1875',
 '3860035_0029 1186875',
 '3860035_0030 10125',
 '3860035_0031 2558375',
 '3860035_0032 1354750',
 '3860035_0033 882',
 '3860035_0034 238243',
 '3860035_0035 750',
 '3860035_0036 250',
 '3860035_0037 141685',
 '3860035_0038 565',
 '3860035_0039 125',
 '3860035_0040 3399